## 小作業
- 請蒐集證交所109年度第一季所有上市公司各產業EPS統計資訊(https://mops.twse.com.tw/mops/web/t163sb19)
- 使用Pandas 整理出資料表格
- 請找出各產業基本每股盈餘(元)的第一名
- 最後將資料儲存於 SQLite 中

###爬取EPS資訊

In [0]:
import requests
payload = {
  'encodeURIComponent': '1',
  'step': '1',
  'firstin': '1',
  'TYPEK': 'sii',
  'year': '109',
  'season': '01'    
}

url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb19'

res = requests.post(url, data = payload)

In [0]:
import pandas
dfs = pandas.read_html(res.text)

In [4]:
len(dfs)

28

In [5]:
type(dfs)

list

In [7]:
df = dfs[0]
df[df['產業別'] != '加權平均數']

,公司代號,公司名稱,產業別,基本每股盈餘(元),普通股每股面額,營業收入,營業利益,營業外收入及支出,稅後淨利
0,1101,台灣水泥股份有限公司,水泥工業,0.55,新台幣 10.0000元,21976200.0,4460154.0,273722.0,3481119.0
1,1109,信大水泥股份有限公司,水泥工業,0.27,新台幣 10.0000元,1165580.0,217284.0,-29077.0,137170.0
2,1102,亞洲水泥股份有限公司,水泥工業,0.24,新台幣 10.0000元,13138882.0,2021639.0,-453351.0,1130599.0
3,1108,幸福水泥股份有限公司,水泥工業,0.23,新台幣 10.0000元,1104720.0,110132.0,-5836.0,93609.0
4,1104,環球水泥股份有限公司,水泥工業,0.07,新台幣 10.0000元,1253758.0,67425.0,-16843.0,48161.0
6,1110,東南水泥股份有限公司,水泥工業,-0.05,新台幣 10.0000元,387944.0,-12597.0,-17552.0,-30941.0
7,1103,嘉新水泥股份有限公司,水泥工業,-0.33,新台幣 10.0000元,495334.0,-158911.0,-46824.0,-214194.0


### 資料預處理

In [0]:
# method 1
industry_ary = []
for industry in dfs:
  industry_ary.append(industry[industry['產業別'] != '加權平均數'])
industry_df = pandas.concat(industry_ary)

In [0]:
# method 2
industry_df = pandas.concat(dfs)
industry_df = industry_df[industry_df['產業別'] != '加權平均數']
industry_df.dropna(inplace=True)

In [13]:
industry_df.head()

,公司代號,公司名稱,產業別,基本每股盈餘(元),普通股每股面額,營業收入,營業利益,營業外收入及支出,稅後淨利
0,1101,台灣水泥股份有限公司,水泥工業,0.55,新台幣 10.0000元,21976200.0,4.46015e+06,273722,3481119.0
1,1109,信大水泥股份有限公司,水泥工業,0.27,新台幣 10.0000元,1165580.0,217284,-29077,137170.0
2,1102,亞洲水泥股份有限公司,水泥工業,0.24,新台幣 10.0000元,13138882.0,2.02164e+06,-453351,1130599.0
3,1108,幸福水泥股份有限公司,水泥工業,0.23,新台幣 10.0000元,1104720.0,110132,-5836,93609.0
4,1104,環球水泥股份有限公司,水泥工業,0.07,新台幣 10.0000元,1253758.0,67425,-16843,48161.0


### 資料統計與分析

In [14]:
# SELECT 產業別, max(基本每股盈餘(元)) from industry_df GROUP BY 產業別;
industry_df.groupby('產業別')['基本每股盈餘(元)'].max()

產業別
光電業         50.10
其他           3.28
其他電子業        5.09
化學工業         2.50
半導體業         8.20
塑膠工業         1.45
建材營造         3.21
橡膠工業         1.55
水泥工業         0.55
汽車工業         6.35
油電燃氣業        0.70
玻璃陶瓷         0.48
生技醫療業        2.62
紡織纖維         3.06
航運業          0.64
觀光事業         1.68
貿易百貨         5.61
資訊服務業        1.86
通信網路業        1.86
造紙工業         0.81
金融保險業        1.00
鋼鐵工業         1.40
電器電纜         2.15
電子通路業        2.51
電子零組件業       5.51
電機機械         3.39
電腦及週邊設備業     8.37
食品工業         1.66
Name: 基本每股盈餘(元), dtype: float64

In [0]:
#  SELECT '公司代號',	'公司名稱', '產業別' '基本每股盈餘(元)_max' FROM
#  industry_df a INNER JOIN
# (SELECT 產業別, max(基本每股盈餘(元) as 基本每股盈餘(元)_max) from industry_df GROUP BY 產業別) b
# ON a.產業別 = b.產業別 

In [19]:
len(industry_df)

929

In [0]:
industry_df['基本每股盈餘(元)_max'] = industry_df.groupby('產業別')['基本每股盈餘(元)'].transform(max)

In [30]:
ret_df

,公司代號,公司名稱,產業別,基本每股盈餘(元),普通股每股面額,營業收入,營業利益,營業外收入及支出,稅後淨利,基本每股盈餘(元)_max
0,1101,台灣水泥股份有限公司,水泥工業,0.55,新台幣 10.0000元,21976200.0,4.46015e+06,273722,3481119.0,0.55
0,1232,大統益股份有限公司,食品工業,1.66,新台幣 10.0000元,4073683.0,327369,13952,273011.0,1.66
0,1325,恒大股份有限公司,塑膠工業,1.45,新台幣 10.0000元,428099.0,160997,3776,135215.0,1.45
0,4439,冠星集團控股有限公司,紡織纖維,3.06,新台幣 10.0000元,1424316.0,117149,10798,96369.0,3.06
0,1530,亞崴機電股份有限公司,電機機械,3.39,新台幣 10.0000元,591116.0,-16718,526915,545391.0,3.39
0,5283,禾聯碩股份有限公司,電器電纜,2.15,新台幣 10.0000元,1196022.0,194156,12771,163490.0,2.15
0,1817,凱撒衛浴股份有限公司,玻璃陶瓷,0.48,新台幣 10.0000元,481608.0,43590,265,34618.0,0.48
0,1904,正隆股份有限公司,造紙工業,0.81,新台幣 10.0000元,9074859.0,1.24925e+06,-126656,897554.0,0.81
0,3004,豐達科技股份有限公司(原宏達),鋼鐵工業,1.40,新台幣 10.0000元,653665.0,86560,-627,73871.0,1.40
0,2114,鑫永銓股份有限公司,橡膠工業,1.55,新台幣 10.0000元,389975.0,132998,4122,109696.0,1.55


In [0]:
ret_df = industry_df[industry_df['基本每股盈餘(元)'] == industry_df.groupby('產業別')['基本每股盈餘(元)'].transform(max)]

### 資料儲存

In [0]:
import sqlite3 as lite
with lite.connect('stock.sqlite') as con:
  industry_df.to_sql('industry_stat', con = con, if_exists='replace')

In [28]:
import sqlite3 as lite
with lite.connect('stock.sqlite') as con:
  df2 = pandas.read_sql_query(
      'SELECT `產業別`,  MAX(`基本每股盈餘(元)`) FROM industry_stat GROUP BY `產業別`;', con = con)
df2.head()

,產業別,MAX(`基本每股盈餘(元)`)
0,光電業,50.10
1,其他,3.28
2,其他電子業,5.09
3,化學工業,2.50
4,半導體業,8.20
